In [8]:
import numpy as np
from scipy.linalg import eigh
from scipy.special import erf

# Чтение данных из POSCAR
def read_poscar(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    scale = float(lines[1])
    lattice_vectors = np.array([[float(x) for x in line.split()] for line in lines[2:5]]) * scale
    atom_types = lines[5].split()
    atom_counts = [int(x) for x in lines[6].split()]
    total_atoms = sum(atom_counts)
    coordinate_lines = lines[8:8+total_atoms]
    coordinates = []
    for line in coordinate_lines:
        try:
            coordinates.append([float(x) for x in line.split()[:3]])  # Берём только координаты x, y, z
        except ValueError:
            continue  # Игнорируем некорректные строки
    coordinates = np.array(coordinates)
    return lattice_vectors, atom_types, atom_counts, coordinates

# Чтение параметров из INCAR
def read_incar(filename):
    params = {}
    with open(filename, 'r') as file:
        for line in file:
            if '=' in line:
                key, value = line.split('=', 1)
                params[key.strip()] = value.strip()
    return params

# 1. Инициализация параметров и базиса для модели (Helium-like)
Z = 2  # Заряд ядра (Helium)
num_basis = 2  # Минимальный базис размерности 2 (STO-3G упрощённо)

def overlap_matrix():
    """Матрица перекрытия S"""
    return np.array([[1.0, 0.2], [0.2, 1.0]])

def kinetic_energy_matrix():
    """Матрица кинетической энергии T"""
    return np.array([[1.0, 0.5], [0.5, 1.0]])

def nuclear_attraction_matrix(Z):
    """Матрица ядерного притяжения V"""
    return -Z * np.array([[1.5, 0.6], [0.6, 1.5]])

def coulomb_repulsion(alpha1, alpha2, r12):
    """Функция для точного расчёта двухэлектронных интегралов"""
    return (2 * np.pi ** (5/2)) / ((alpha1 + alpha2) * np.sqrt(alpha1 + alpha2)) * erf(np.sqrt(alpha1 + alpha2) * r12)

def two_electron_integrals_exact(basis_centers, alpha):
    """Точные интегралы двухэлектронного взаимодействия для s-орбиталей"""
    num_basis = len(basis_centers)
    g = np.zeros((num_basis, num_basis, num_basis, num_basis))
    for mu in range(num_basis):
        for nu in range(num_basis):
            for lam in range(num_basis):
                for sigma in range(num_basis):
                    r12 = np.linalg.norm(basis_centers[mu] - basis_centers[lam])
                    g[mu, nu, lam, sigma] = coulomb_repulsion(alpha[mu], alpha[lam], r12)
    return g

def hartree_fock(Z, basis_centers, alpha, num_iter=50, tol=1e-6):
    """Основная функция Hartree-Fock"""
    S = overlap_matrix()
    T = kinetic_energy_matrix()
    V = nuclear_attraction_matrix(Z)
    g = two_electron_integrals_exact(basis_centers, alpha)

    # Начальные параметры
    H_core = T + V  # Core Hamiltonian
    D = np.zeros((num_basis, num_basis))  # Матрица плотности

    # Ортогонализация базиса
    S_inv_sqrt = np.linalg.inv(np.sqrt(S))

    E_total_old = 0.0

    for iteration in range(num_iter):
        # 1. Построение Fock-матрицы
        F = H_core.copy()
        for mu in range(num_basis):
            for nu in range(num_basis):
                F[mu, nu] += np.sum(D * (2 * g[mu, nu] - g[mu, :, :, nu]))
        
        # 2. Решение уравнений Рёлиха
        F_prime = S_inv_sqrt @ F @ S_inv_sqrt
        eigvals, eigvecs = eigh(F_prime)
        C = S_inv_sqrt @ eigvecs  # Коэффициенты МО

        # 3. Обновление матрицы плотности
        C_occ = C[:, :1]  # Только занятая орбиталь
        D_new = np.einsum('pi,qi->pq', C_occ, C_occ)

        # 4. Вычисление энергии
        E_electronic = np.sum(D_new * (H_core + F))
        E_total = E_electronic + Z**2 / 2  # Приближённая энергия ядра
        
        # Проверка сходимости
        if abs(E_total - E_total_old) < tol:
            print(f"Сошлось за {iteration+1} итераций. Общая энергия: {E_total:.6f} Hartree")
            return E_total

        E_total_old = E_total
        D = D_new

    print("Не сошлось за заданное число итераций.")
    return E_total

# Запуск кода с POSCAR и INCAR
poscar_file = 'POSCAR'
incar_file = 'INCAR'

lattice_vectors, atom_types, atom_counts, coordinates = read_poscar(poscar_file)
params = read_incar(incar_file)

print("Параметры из INCAR:", params)
print("Координаты атомов:", coordinates)

# Выполнение Hartree-Fock для заряда ядра из INCAR
Z = int(params.get('Z', 2))
alpha = [0.5 for _ in coordinates]  # Пример экспоненциальных параметров базиса (s-орбитали)
E_HF = hartree_fock(Z, coordinates, alpha)
print(f"Энергия Хартри-Фока: {E_HF:.6f} Hartree")


Параметры из INCAR: {'ALGO': 'Fast', 'EDIFF': '1E-5', 'EINT': '-0.542', 'ENCUT': '600', 'ISMEAR': '0', 'ISTART': '1', 'LCHARG': '.FALSE.', 'LORBIT': '11', 'LPARD': '.TRUE.', 'LSEPB': '.FALSE.', 'LSEPK': '.FALSE.', 'LSORBIT': '.TRUE.', 'LWAVE': '.FALSE.', 'NBMOD': '-3', 'NEDOS': '5000', 'PREC': 'Accurate', 'SIGMA': '0.01'}
Координаты атомов: [[ 0.333334  0.666666  0.097538]
 [ 0.       -0.        0.097538]]
Сошлось за 2 итераций. Общая энергия: -6.508610 Hartree
Энергия Хартри-Фока: -6.508610 Hartree


In [7]:
import numpy as np
from scipy.linalg import eigh
from ase import Atoms
from jarvis.db.figshare import data

# Параметры для H_core
alpha_params = {'C': -7.441409}  # Энергия на диагонали для атома углерода
beta_params = {('C', 'C'): -2.135484}  # Внедиагональные элементы для связи C-C
gamma_params = {}  # Пустой словарь для дополнительных параметров (если нужны)

def calculate_h_core(atoms, alpha_params, beta_params, gamma_params):
    """Функция для построения матрицы H_core."""
    num_atoms = len(atoms)
    H_core = np.zeros((num_atoms, num_atoms))
    for i, atom_i in enumerate(atoms.get_chemical_symbols()):
        H_core[i, i] = alpha_params.get(atom_i, 0.0)  # Диагональные элементы
        for j, atom_j in enumerate(atoms.get_chemical_symbols()):
            if i != j:
                H_core[i, j] = beta_params.get((atom_i, atom_j), 0.0)  # Внедиагональные элементы
    return H_core

def hartree_fock_pbc(S, H_core, num_electrons, k_points):
    """Вычисление энергии Hartree-Fock с периодическими границами."""
    total_energy = 0.0
    for k in k_points:
        Fock_matrix = H_core * np.exp(1j * k)
        eigvals, _ = eigh(Fock_matrix, S)
        total_energy += np.sum(eigvals[:num_electrons // 2])
    return total_energy.real / len(k_points)

# Получаем данные из JARVIS
dft_2d_data = data('dft_2d')
material_data = None

# Поиск графена (пример JVASP-60368)
for material in dft_2d_data:
    if material['jid'] == 'JVASP-60368':
        material_data = material
        break

if material_data:
    # Инициализация атомов и параметров
    atoms_data = material_data['atoms']
    atoms = Atoms(symbols=atoms_data['elements'],
                  positions=atoms_data['coords'],  # Исправляем ключ 'positions' на 'coords'
                  cell=atoms_data['lattice_mat'],
                  pbc=True)
    S = np.eye(len(atoms))  # Матрица перекрытий (единичная)
    
    # Построение H_core
    H_core = calculate_h_core(atoms, alpha_params, beta_params, gamma_params)
    print("H_core матрица:\n", H_core)
    
    # k-точки для расчёта PBC
    k_points = [0.0, np.pi / 2, np.pi]
    hf_energy = hartree_fock_pbc(S, H_core, num_electrons=2, k_points=k_points)
    
    # JARVIS DFT энергия
    dft_energy = material_data.get('optb88vdw_total_energy', None)
    
    print(f"Hartree-Fock энергия: {hf_energy:.6f} эВ")
    print(f"JARVIS-DFT энергия: {dft_energy:.6f} эВ")
else:
    print("Материал графен (JVASP-60368) не найден в базе данных JARVIS.")


Obtaining 2D dataset 1.1k ...
Reference:https://www.nature.com/articles/s41524-020-00440-1
Other versions:https://doi.org/10.6084/m9.figshare.6815705
Loading the zipfile...
Loading completed.
H_core матрица:
 [[-7.441409 -2.135484]
 [-2.135484 -7.441409]]
Hartree-Fock энергия: -2.135484 эВ
JARVIS-DFT энергия: -7.956340 эВ
